In [1]:
import sys
sys.path.append('../')

In [2]:
from final_utils import *
from ResNet import *
from ResNet_FC import *
import math 

In [3]:
#idea
# fit_generator()
# gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
# session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
# gpu = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(gpu[0], True)


In [4]:
gpus = tf.config.list_physical_devices('GPU')
# gpu = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[1], True)
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.set_visible_devices(gpus[1], 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)


2 Physical GPUs, 1 Logical GPU


2022-12-16 20:09:23.713271: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 20:09:23.713594: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 20:09:23.717387: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 20:09:23.717693: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 20:09:23.718282: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

In [5]:
root_path = generate_root_path()

### DataLoader and other nesseccary functions

In [6]:
def lr_exp_decay(epoch, lr):
    k = 0.04
    return lr * np.exp(-k*epoch)

def make_dataset(paths, batch_size):
    main_path = f'{root_path}Data/AesthAI/alm/splitted/alm_train/'
    def parse_image(filename):
        image = tf.io.read_file(filename)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [996, 996]) #ToDo: check this, why we will reizezi img if we load alread resized img
        return image

    def configure_for_performance(ds):
        ds = ds.shuffle(buffer_size=1000)
        ds = ds.batch(batch_size)
        ds = ds.repeat()
        ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
        return ds

    labels = [np.load(main_path + '/predictions/mg_cnn_fc/' + name.split('.')[0].split('/')[-1] + '.npy') for name in paths]
    filenames_ds = tf.data.Dataset.from_tensor_slices(paths)
    images_ds = filenames_ds.map(parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    labels_ds = tf.data.Dataset.from_tensor_slices(labels)
    ds = tf.data.Dataset.zip((images_ds, labels_ds))
    ds = configure_for_performance(ds)

    return ds

### Trainer

In [7]:
def trainer(model, weights_path, train_paths, val_paths, batch_size=128, epochs=30, learning_rate=0.03, verbose=0):
    data = make_dataset(train_paths, batch_size)
    data_val = make_dataset(val_paths, batch_size)
    checkpoint = keras.callbacks.ModelCheckpoint(weights_path, monitor='val_loss', verbose=1, save_best_only=True, 
                                                 mode='min')
    schedule = tf.keras.callbacks.LearningRateScheduler(lr_exp_decay, verbose=0)
    callbacks_list = [checkpoint, schedule]
    history = model.fit(data, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=data_val, 
                        steps_per_epoch=math.ceil(len(train_paths)/32), validation_steps=math.ceil(len(train_paths)/32), 
                        validation_batch_size=batch_size, callbacks=callbacks_list)

    return history

### Loading data

In [8]:
main_path = f'{root_path}Data/AesthAI/alm/splitted/alm_train/'
paths_bad = []
paths_good = []
    
for i in range(7):
    alm_train_bad = open(f'{main_path}data_bad{i+1}.json')
    bad_data = json.load(alm_train_bad)
    
    for data in bad_data:
        path_to_img = main_path + f'images/{data["label"]}/{data["splitted"]}_resized_996x996/' + data['name']
        paths_bad.append(path_to_img)
        
alm_train_good = open(f'{main_path}/data_good1.json')
good_data = json.load(alm_train_good)
for data in good_data:
    path_to_img = main_path + f'images/{data["label"]}/{data["splitted"]}_resized_996x996/' + data['name']
    paths_good.append(path_to_img)
    
for i in range(7):
    paths_bad[i] = np.squeeze(np.array(paths_bad[i]))
paths_good = np.squeeze(np.array(paths_good))
   
# Generating static validation data
paths_bad, paths_bad_val = extract_static_val_data(paths_bad, perc = 0.014) #original - 0.017
paths_good, paths_good_val = extract_static_val_data(paths_good, perc = 0.06) #original - 0.11

paths_bad = np.array(paths_bad)
paths_bad_val = np.array(paths_bad_val)
paths_good = np.array(paths_good)
paths_good_val = np.array(paths_good_val)

In [9]:
full_data = np.concatenate((np.repeat(paths_good, 7), paths_bad))
    
#shuffling
idx = np.random.permutation(len(full_data))
full_data = full_data[idx]
# full_data = full_data[:5000]  # debug
full_data.shape
paths_val = np.concatenate((paths_bad_val, paths_good_val ) , axis=0)

### Creating model and training

In [10]:
batch_size = 32 #64
epochs = 20
learning_rate = 0.009

In [11]:
def fc_model_softmax(input_num=16928):
    input_ = Input(shape=(input_num,))
    x = Dense(1024, kernel_initializer='he_normal', activation='relu')(input_)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Dense(256, kernel_initializer='he_normal', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    pred = Dense(2, activation='softmax')(x)
    model = Model(input_,pred)
    return model

In [12]:
# with tf.device('/GPU:0'):
model_res = ResNet18(num_classes=5000)
model_fc = fc_model_softmax(input_num=5000)
model = resnet_fc(model_res, model_fc)
model.build((None, 996, 996, 3))

weights_path = '../models/Resnet/resnet_fc_996_probs/'
# model.save_weights(weights_path)
model.summary()
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=keras.optimizers.Adam(learning_rate=learning_rate))

Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model: "resnet_fc"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 res_net18 (ResNet18)        multiple                  4074504   
                                                                 
 model (Functional)          (None, 2)                 5389058   
                                                                 
Total params: 9,463,562
Trainable params: 9,456,522
Non-trainable params: 7,040
_________________________________________________________________


In [13]:
 history = trainer(model, weights_path, full_data, paths_val, batch_size=batch_size, epochs=epochs, learning_rate=learning_rate, 
                   verbose=1)    

Epoch 1/20


2022-12-16 20:09:49.631520: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-12-16 20:09:50.087587: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-12-16 20:09:50.089138: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-12-16 20:09:50.089274: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-12-16 20:09:50.090816: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-12-16 20:09:50.091317: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-12-16 20:09:51.356287: I tensorflow/stream_executor/cuda/c

4597/4597 [==============================] - ETA: 0s - loss: 0.0931
Epoch 1: val_loss improved from inf to 0.19017, saving model to ../models/Resnet/resnet_fc_996_probs/


INFO:tensorflow:Assets written to: ../models/Resnet/resnet_fc_996_probs/assets


INFO:tensorflow:Assets written to: ../models/Resnet/resnet_fc_996_probs/assets


...

INFO:tensorflow:Assets written to: ../models/Resnet/resnet_fc_996_probs/assets


4597/4597 [==============================] - 4178s 909ms/step - loss: 0.0347 - val_loss: 0.0663 - lr: 0.0021
Epoch 10/20
4597/4597 [==============================] - ETA: 0s - loss: 0.0322
Epoch 10: val_loss improved from 0.06634 to 0.06251, saving model to ../models/Resnet/resnet_fc_996_probs/


INFO:tensorflow:Assets written to: ../models/Resnet/resnet_fc_996_probs/assets


INFO:tensorflow:Assets written to: ../models/Resnet/resnet_fc_996_probs/assets


4597/4597 [==============================] - 4230s 920ms/step - loss: 0.0322 - val_loss: 0.0625 - lr: 0.0015
Epoch 11/20
4597/4597 [==============================] - ETA: 0s - loss: 0.0304
Epoch 11: val_loss did not improve from 0.06251
4597/4597 [==============================] - 4088s 889ms/step - loss: 0.0304 - val_loss: 0.2268 - lr: 9.9723e-04
Epoch 12/20
4597/4597 [==============================] - ETA: 0s - loss: 0.0281
Epoch 12: val_loss did not improve from 0.06251
4597/4597 [==============================] - 4068s 885ms/step - loss: 0.0281 - val_loss: 0.0725 - lr: 6.4225e-04
Epoch 13/20
4597/4597 [==============================] - ETA: 0s - loss: 0.0268
Epoch 13: val_loss did not improve from 0.06251
4597/4597 [==============================] - 4128s 898ms/step - loss: 0.0268 - val_loss: 0.0900 - lr: 3.9741e-04
Epoch 14/20
4597/4597 [==============================] - ETA: 0s - loss: 0.0255
Epoch 14: val_loss did not improve from 0.06251
4597/4597 [=============================